# Setup and install for test13

In [ ]:

import os
import re
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%pip install gpflow
%pip install plotnine
%pip install plotly
%pip install keras_self_attention
%pip install keras_tuner
%pip install pycm

In [ ]:
from plotnine import *
from plotnine.themes import *
import plotly as py
import plotly.graph_objs as go
import pandas as pd


In [ ]:
import tensorflow as tf
from scipy.io import loadmat
import random
import math
import tensorflow_probability as tfp

_paper_name_ establishes the reusable name of the paper, it represents the directory under data_papers on the google drive

In [ ]:
paper_name = "test13"

In [ ]:
import os, sys
import errno

# make a directory if it does not exist
def make_dir_if_not_exist(used_path):
    if not os.path.isdir(used_path):
        try:
            os.mkdir(used_path)
        except OSError as exc:
            if exc.errno != errno.EEXIST:
                raise exc
            else:
                raise ValueError(f'{used_path} directoy cannot be created because its parent directory does not exist.')

# make directories if they do not exist

make_dir_if_not_exist("/content/drive/MyDrive/data_papers/")
make_dir_if_not_exist(f"/content/drive/MyDrive/data_papers/{paper_name}")
make_dir_if_not_exist(f"/content/drive/MyDrive/data_papers/{paper_name}/model_features/")
make_dir_if_not_exist(f"/content/drive/MyDrive/data_papers/{paper_name}/model_checkpoints/")
make_dir_if_not_exist(f"/content/drive/MyDrive/data_papers/{paper_name}/model_history/")
make_dir_if_not_exist(f"/content/drive/MyDrive/data_papers/{paper_name}/model_finals/")
make_dir_if_not_exist(f"/content/drive/MyDrive/data_papers/{paper_name}/gp_collab/")
make_dir_if_not_exist(f"/content/drive/MyDrive/data_papers/{paper_name}/model_predictions/")
make_dir_if_not_exist(f"/content/drive/MyDrive/data_papers/{paper_name}/model_ccs/")
make_dir_if_not_exist(f"/content/drive/MyDrive/data_papers/{paper_name}/summary_results/")
make_dir_if_not_exist(f"/content/drive/MyDrive/data_papers/{paper_name}/summary_results/temp/")



In [ ]:
data_location = f'/content/drive/MyDrive/data_papers/{paper_name}'

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import  Dense, Flatten, Activation, Dropout, Embedding, Conv1D, MaxPooling1D, GRU
from tensorflow.keras.layers import LSTM, TimeDistributed, Permute,Reshape, Lambda, RepeatVector, Input,Multiply
from tensorflow.keras.layers import  Dense, Flatten, Activation, Dropout, Embedding, Conv1D, Conv2D, MaxPooling2D, MaxPooling1D, Concatenate, BatchNormalization, GaussianNoise
from tensorflow.keras.layers import SimpleRNN, GRU, LeakyReLU
from tensorflow.keras.layers import Concatenate, Average 
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import  Bidirectional
from timeit import default_timer as timer
import h5py as h5
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
import errno
from tensorflow.keras.applications.imagenet_utils import decode_predictions
import random
import warnings
import gpflow
from gpflow.utilities import ops, print_summary, set_trainable
from gpflow.config import set_default_float, default_float, set_default_summary_fmt
from gpflow.ci_utils import ci_niter
import warnings
from functools import partial
from multiprocessing import Pool, cpu_count
from tensorflow.keras.layers import  Dense, Flatten, Activation, Dropout, Embedding, Conv1D, Conv2D, MaxPooling2D, MaxPooling1D, Concatenate, BatchNormalization, GaussianNoise
from tensorflow.keras.layers import LSTM, TimeDistributed, Permute, Reshape, Lambda, RepeatVector, Input, Multiply, SimpleRNN, GRU, LeakyReLU
from keras_self_attention import SeqSelfAttention, SeqWeightedAttention
from tensorflow.keras import regularizers
import os
from collections import defaultdict
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
import keras_tuner as kt

In [ ]:
# Run this cell to load the dataset
INPUT_DIM = 8    # 

hf_Train = h5.File(f'{data_location}/Fold_10_Train_Data_1000.h5', 'r')
hf_Test = h5.File(f'{data_location}/Fold_10_Test_Data_1000.h5', 'r')

X_train = hf_Train['Train_Data'] # Get train set
X_train = np.array(X_train)
Y_train = hf_Train['Label']      # Get train label
Y_train = np.array(Y_train)

X_test = hf_Test['Train_Data']     # Get test set
X_test = np.array(X_test)
Y_test = hf_Test['Label']       # Get test label
Y_test = np.array(Y_test)

Y_train = to_categorical(Y_train, 13)  # Process the label of tain
Y_test = to_categorical(Y_test, 13)    #  Process the label of te

def coShuffled_vectors(X,Y):
    if tf.shape(X)[0] ==  tf.shape(Y)[0]:
        test_idxs = tf.range(start=0, limit=tf.shape(X)[0], dtype=tf.int32)
        shuffled_test_idxs = tf.random.shuffle(test_idxs)
        return ( tf.gather(X, shuffled_test_idxs), tf.gather(Y, shuffled_test_idxs) )
    else:
        raise ValueError(f"0-dimension has to be the same {tf.shape(X)[0]} != {tf.shape(Y)[0]}")

X_test_shuffled, Y_test_shuffled = coShuffled_vectors(np.array(hf_Test['Train_Data']), to_categorical(np.array(hf_Test['Label']),13))
X_train_shuffled, Y_train_shuffled = coShuffled_vectors(np.array(hf_Train['Train_Data']), to_categorical(np.array(hf_Train['Label']),13))



In [ ]:
def plot_history(history):
    acc_keys = [k for k in history.history.keys() if k in ('accuracy', 'val_accuracy')] 
    loss_keys = [k for k in history.history.keys() if not k in acc_keys]
    for k, v in history.history.items():
        if k in acc_keys:
            plt.figure(1)
            plt.plot(v)
        else:
            plt.figure(2)
            plt.plot(v)
    plt.figure(1)
    plt.title('Accuracy vs. epochs')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(acc_keys, loc='upper right')
    plt.figure(2)
    plt.title('Loss vs. epochs')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(loss_keys, loc='upper right')
    plt.show()



In [ ]:

def get_layer_by_name(layers, name, return_first = True):
    matching_named_layers = [ l for l in layers if l.name==name]
    if not matching_named_layers:
        return None
    return matching_named_layers[0] if return_first else matching_named_layers


def get_combined_features_from_models(
        to_combine, 
        X_train, Y_train, 
        X_test, Y_test,
        reverse_one_hot = False,
        normalize_X_func = None):
    
    models = dict()
    X_trains_out = []
    X_test_out = []
    XY_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: None))))
    
    if reverse_one_hot:
        Y_train_new = np.apply_along_axis(np.argmax, 1, Y_train) + 1  
        Y_test_new = np.apply_along_axis(np.argmax, 1, Y_test) + 1 
    else:
        Y_train_new = Y_train.copy()
        Y_test_new = Y_test.copy()
    
    for model_file_name, layer_name, kwargs in to_combine:
        model_here = None
        if isinstance(model_file_name, tf.keras.models.Model):
            model_here = model_file_name
            model_file_name = model_here.name
        else:
            if model_file_name in models.keys():
                model_here = models[model_file_name]
            else:
                model_here = tf.keras.models.load_model(model_file_name, **kwargs) if kwargs is not None else tf.keras.models.load_model(model_file_name)
        features_model = Model(model_here.input, 
                               get_layer_by_name(model_here.layers,layer_name).output)
        if normalize_X_func is None:
            X_trains_out.append(np.array(features_model.predict(X_train), dtype='float64'))
            X_test_out.append(np.array(features_model.predict(X_test), dtype='float64'))
        else:
            X_trains_out.append(np.array(normalize_X_func(features_model.predict(X_train)), dtype='float64'))
            X_test_out.append(np.array(normalize_X_func(features_model.predict(X_test)), dtype='float64'))
        XY_dict[model_file_name][layer_name]['Train']['X'] = X_trains_out[-1]
        XY_dict[model_file_name][layer_name]['Test']['X'] = X_test_out[-1]
        XY_dict[model_file_name][layer_name]['Train']['Y'] = Y_train_new
        XY_dict[model_file_name][layer_name]['Test']['Y'] = Y_test_new
        models[model_file_name] = model_here            
        
    X_train_new = np.concatenate(tuple(X_trains_out), axis = 1)
    X_test_new = np.concatenate(tuple(X_test_out), axis = 1)
    
    data_train = (X_train_new, Y_train_new)
    data_test = (X_test_new, Y_test_new)    
    
        
    return (models, data_train, data_test,XY_dict)



In [ ]:
def run_and_save_model(model_func, X_train, Y_train, kwargs):    
    m = model_func()
    m.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=['accuracy'])
    history = m.fit(X_train, Y_train, **kwargs) 
    m.save(f"{m.name}_Tenth_Fold_New_Model_500_8") #Save the model


In [ ]:
from pycm import ConfusionMatrix

In [ ]:
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import h5py as h5
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv1D, LeakyReLU, MaxPooling1D, BatchNormalization, GaussianNoise, Dropout, Dense, Flatten
import errno
import os
from collections import defaultdict
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.summary import create_file_writer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import kerastuner as kt
from kerastuner import HyperModel
import numpy as np
import itertools
import multiprocessing
from numpy import genfromtxt
import seaborn as sns
from sklearn.metrics import precision_recall_fscore_support
from pycm import ConfusionMatrix

In [ ]:
data_location

'/content/drive/MyDrive/data_papers/test13'

In [ ]:

def coShuffled_vectors(X, Y):
    if tf.shape(X)[0] == tf.shape(Y)[0]:
        test_idxs = tf.range(start=0, limit=tf.shape(X)[0], dtype=tf.int32)
        shuffled_test_idxs = tf.random.shuffle(test_idxs)
        return (tf.gather(X, shuffled_test_idxs), tf.gather(Y, shuffled_test_idxs))
    else:
        raise ValueError(f"0-dimension has to be the same {tf.shape(X)[0]} != {tf.shape(Y)[0]}")

def reverse_one_hot(Y_input):
    return np.apply_along_axis(np.argmax, 1, Y_input) + 1

def getNpArrayFromH5(hf_Data):
    X_train = hf_Data['Train_Data']  # Get train set
    X_train = np.array(X_train)
    Y_train = hf_Data['Label']  # Get train label
    Y_train = np.array(Y_train)
    return X_train, Y_train

def get_confusion_matrix_classification(model, X, Y_true):
    y_pred = model.predict(X)
    y_true = np.apply_along_axis(np.argmax, 1, Y_true)
    y_pred = np.apply_along_axis(np.argmax, 1, y_pred)
    return (confusion_matrix(y_true, y_pred), y_pred, y_true)

def misclass_perc_to_weight(input_confusion, add_base=True, func=None):
    perc_misclassified = 1.0 - np.array([ input_confusion[x,x] for x in np.arange(input_confusion.shape[0]).tolist() ])/input_confusion.sum(axis=1)
    
    base_val = min(perc_misclassified[perc_misclassified>0.0])
    if add_base:        
        perc_misclassified = perc_misclassified + base_val
    
    perc_misclassified = [ x/base_val for x in perc_misclassified]
    return dict([ (idx, func(perc_val)) if func is not None else (idx, perc_val) for idx, perc_val in enumerate(perc_misclassified) ])

def prf(model,xtest, ytest):
  y_pred = np.apply_along_axis(np.argmax, 1, model.predict(xtest))
  y_true = np.apply_along_axis(np.argmax, 1, ytest)
  return precision_recall_fscore_support(y_true, y_pred, average="weighted")

def get_sp_pr_rc_f1(model,xtest, ytest):  
    y_pred = np.apply_along_axis(np.argmax, 1, model.predict(xtest))
    y_true = np.apply_along_axis(np.argmax, 1, ytest)
    cmres = ConfusionMatrix(actual_vector=y_true,predict_vector=y_pred)
    pr, rc, f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")   
    return cmres.TNR_Macro, pr, rc, f1
 
def get_sp_pr_rc_f1_acc(model,xtest, ytest):  
    spec, pr, rc, f1 = get_sp_pr_rc_f1(model,xtest, ytest)
    acc = model.evaluate(xtest,ytest)[-1]        
    return spec, pr, rc, f1, acc



In [ ]:
def plot_history(history):
    acc_keys = [k for k in history.history.keys() if k in ('accuracy', 'val_accuracy')]
    loss_keys = [k for k in history.history.keys() if not k in acc_keys]
    for k, v in history.history.items():
        if k in acc_keys:
            plt.figure(1)
            plt.plot(v)
        else:
            plt.figure(2)
            plt.plot(v)
    plt.figure(1)
    plt.title('Accuracy vs. epochs')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(acc_keys, loc='lower right')
    plt.figure(2)
    plt.title('Loss vs. epochs')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(loss_keys, loc='upper right')
    plt.show()


def get_layer_by_name(layers, name, return_first=True):
    matching_named_layers = [l for l in layers if l.name == name]
    if not matching_named_layers:
        return None
    return matching_named_layers[0] if return_first else matching_named_layers


def make_dir_if_not_exist(used_path):
    if not os.path.isdir(used_path):
        try:
            os.mkdir(used_path)
        except OSError as exc:
            if exc.errno != errno.EEXIST:
                raise exc
            else:
                raise ValueError(f'{used_path} directoy cannot be created because its parent directory does not exist.')


def source_model(model_func, model_name, input_shape):
    m = None
    if isinstance(model_func, tf.keras.models.Model):
        m = model_func
        m._name = model_name
    else:
        m = model_func(model_name, input_shape)
    return m


In [ ]:


def compile_and_fit_model_with_tb(model_func,
                                  model_name,
                                  input_shape,
                                  X_train,
                                  Y_train,
                                  save_every_epoch=True,
                                  save_final=False,
                                  **kwargs):
    m = None
    if isinstance(model_func, tf.keras.models.Model):
        m = model_func
        m._name = model_name
    else:
        m = model_func(model_name, input_shape)
    tb_callback = TensorBoard(log_dir=f'{m.name}_logs', histogram_freq=kwargs.pop("histogram_freq", 1))
    if save_every_epoch:
        tb_callback.append(ModelCheckpoint(f'{m.name}' + '_model_{epoch:03d}_{val_accuracy:0.2f}'))
    m.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    history = m.fit(X_train, Y_train, callbacks=[tb_callback], verbose=2, **kwargs)
    if save_final:
        make_dir_if_not_exist(model_name)
        m.save(f"{m.name}_saved_model_after_fit")  # Save the model
    return (m, history)
    # m.save(f"{m.name}_Tenth_Fold_New_Model_500_8") #Save the model


def compile_model_and_fit_with_custom_loop(model_func,
                                           model_name,
                                           input_shape,
                                           X_train,
                                           Y_train,
                                           **kwargs):
    make_dir_if_not_exist(model_name)
    m = None
    if isinstance(model_func, tf.keras.models.Model):
        m = model_func
        m._name = model_name
    else:
        m = model_func(model_name, input_shape)

    train_writer = create_file_writer(f'{m.name}_logs/train/')
    test_writer = create_file_writer(f'{m.name}_logs/test/')
    train_step = test_step = 0

    acc_metric = tf.keras.metrics.CategoricalAccuracy()
    optimizer = tf.keras.optimizers.Adam()
    num_epochs = kwargs.get("epochs", 10)

    AUTOTUNE = tf.data.experimental.AUTOTUNE
    BATCH_SIZE = kwargs.get("batch_size", 32)
    X_test, Y_test = kwargs.get("validation_data", (None, None))
    if X_test is None:
        raise ValueError("Missing X validation data")
    if Y_test is None:
        raise ValueError("Missing Y validation data")

    train_dataset_tf = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
    train_dataset_tf = train_dataset_tf.batch(BATCH_SIZE)
    train_dataset_tf = train_dataset_tf.prefetch(AUTOTUNE)

    test_dataset_tf = tf.data.Dataset.from_tensor_slices((X_test, Y_test))
    test_dataset_tf = train_dataset_tf.batch(BATCH_SIZE)
    test_dataset_tf = train_dataset_tf.prefetch(AUTOTUNE)

    loss_fn = tf.keras.losses.CategoricalCrossentropy()

    for epoch in range(num_epochs):
        # Iterate through training set
        for batch_idx, (x, y) in enumerate(train_dataset_tf):
            with tf.GradientTape() as tape:
                y_pred = m(x, training=True)
                loss = loss_fn(y, y_pred)

            gradients = tape.gradient(loss, m.trainable_weights)
            optimizer.apply_gradients(zip(gradients, m.trainable_weights))
            acc_metric.update_state(y, y_pred)

            with train_writer.as_default():
                tf.summary.scalar("Loss", loss, step=train_step)
                tf.summary.scalar(
                    "Accuracy", acc_metric.result(), step=train_step,
                )
                train_step += 1
        # Reset accuracy in between epochs (and for testing and test)
        acc_metric.reset_states()
        # Iterate through test set
        for batch_idx, (x, y) in enumerate(test_dataset_tf):
            y_pred = m(x, training=False)
            loss = loss_fn(y, y_pred)
            acc_metric.update_state(y, y_pred)
            with test_writer.as_default():
                tf.summary.scalar("Loss", loss, step=test_step)
                tf.summary.scalar(
                    "Accuracy", acc_metric.result(), step=test_step,
                )
                test_step += 1

        acc_metric.reset_states()  # Reset accuracy in between epochs (and for testing and test)

    return m



def run_mirrored_strategy(model_func, base_batch_size, nepochs, x_train, y_train, x_test, y_test, **kwargs):
    strategy = tf.distribute.MirroredStrategy()
    with strategy.scope():
        model = model_func()
        model.compile(
            optimizer=tf.keras.optimizers.Adam(),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=tf.keras.metrics.SparseCategoricalAccuracy()
        )
    batch_size_mirr_strat = base_batch_size * strategy.num_replicas_in_sync
    history = model.fit(x_train, y_train, epochs=nepochs, batch_size=batch_size_mirr_strat,
                        validation_data=(x_test, y_test),
                        **kwargs)
    return model, history


def sparse_setdiff(a1, a2):
    a1a = a1.reshape(a1.shape[0], -1)
    a2a = a2.reshape(a2.shape[0], -1)
    spa2a = [np.where(x)[0].tolist() for x in a2a]
    spa1a = [np.where(x)[0].tolist() for x in a1a]
    idxs_to_keep = []
    for idx, sample in enumerate(spa1a):
        try:
            spa2a.index(sample)
        except ValueError:
            # not in list
            idxs_to_keep.append(idx)
    return a1[idxs_to_keep], idxs_to_keep



def reinitialize_weights(model):
    for ix, layer in enumerate(model.layers):
        if hasattr(model.layers[ix], 'kernel_initializer') and hasattr(model.layers[ix], 'bias_initializer'):
            weight_initializer = model.layers[ix].kernel_initializer
            bias_initializer = model.layers[ix].bias_initializer
    
            old_weights, old_biases = model.layers[ix].get_weights()
    
            model.layers[ix].set_weights([
                weight_initializer(shape=old_weights.shape),
                bias_initializer(shape=len(old_biases))])            
    return model

def reverse_tensor(X):
    return tf.gather(X, tf.reverse(tf.range(start=0, limit=tf.shape(X)[0], dtype=tf.int32),(0,)) )

def get_combined_features_from_models(
    
        to_combine,
        X_train, Y_train,
        X_test, Y_test,
        reverse_one_hot=False,
        normalize_X_func=None):
    
    models = []
    models_dict = {}
    X_trains_out = []
    X_test_out = []
    XY_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: None))))

    models_have_different_inputs = isinstance(Y_train,list)

    if reverse_one_hot:
        if models_have_different_inputs:
            Y_train_new = np.apply_along_axis(np.argmax, 1, Y_train) + 1
            Y_test_new = np.apply_along_axis(np.argmax, 1, Y_test) + 1
        else:
            Y_train_new = [ np.apply_along_axis(np.argmax, 1, y_train) + 1 for y_train in Y_train ]  
            Y_test_new = [ np.apply_along_axis(np.argmax, 1, y_test) + 1 for y_test in Y_test ]              
    else:
        if models_have_different_inputs:
            Y_train_new = Y_train.copy()
            Y_test_new = Y_test.copy()
        else:
            Y_train_new = [ y_train.copy() for y_train in Y_train ] 
            Y_test_new = [ y_test.copy() for y_test in Y_train ] 
            

    extraction_counter =0
    for model_file_name, layer_name, kwargs in to_combine:
        model_here = None
        if isinstance(model_file_name, tf.keras.models.Model):
            model_here = model_file_name
            model_file_name = model_here.name
        else:
            if model_file_name in models_dict.keys():
                model_here = models_dict[model_file_name]
            else:
                model_here = tf.keras.models.load_model(model_file_name,
                                                        **kwargs) if kwargs is not None else tf.keras.models.load_model \
                    (model_file_name)

        features_model = Model(model_here.input,
                               get_layer_by_name(model_here.layers, layer_name).output)
        
        if normalize_X_func is None:
            X_trains_out.append(np.array(features_model.predict(X_train if not models_have_different_inputs else X_train[extraction_counter]), dtype='float64'))
            X_test_out.append(np.array(features_model.predict(X_test if not models_have_different_inputs else X_test[extraction_counter]), dtype='float64'))
        else:
            X_trains_out.append(np.array(normalize_X_func(features_model.predict(X_train if not models_have_different_inputs else X_train[extraction_counter])), dtype='float64'))
            X_test_out.append(np.array(normalize_X_func(features_model.predict(X_test if not models_have_different_inputs else X_test[extraction_counter])), dtype='float64'))
        XY_dict[model_file_name][layer_name]['Train']['X'] = X_trains_out[-1]
        XY_dict[model_file_name][layer_name]['Test']['X'] = X_test_out[-1]
        XY_dict[model_file_name][layer_name]['Train']['Y'] = Y_train_new
        XY_dict[model_file_name][layer_name]['Test']['Y'] = Y_test_new
        models.append(((model_file_name, layer_name), (model_here, features_model)))
        models_dict[model_file_name] = model_here
        extraction_counter += 1

    X_train_new = np.concatenate(tuple(X_trains_out), axis=1)
    X_test_new = np.concatenate(tuple(X_test_out), axis=1)

    data_train = (X_train_new, Y_train_new)
    data_test = (X_test_new, Y_test_new)

    return models, data_train, data_test, XY_dict

class SaveBestOverCombinedThresholds(tf.keras.callbacks.Callback):

    def __init__(self, colab_download = False, observed_values = [ ('accuracy',0.9) ] ):
        self.thresholds = dict(observed_values)
        self.last_best_values = dict([ (obs_name, np.nan) for obs_name in self.thresholds.keys()] )
        self.colab_download = colab_download
        
    def on_epoch_end(self, epoch, logs=None):        
        register = None
        for k,v in self.thresholds.items():
            if k not in logs.keys():
                raise ValueError(f"{k} not found in logs")
            passes_threshold = logs[k] > self.thresholds[k]                 
            register = passes_threshold if register is None else (register and passes_threshold)
        
        if register:
            for k,v in self.thresholds.items():
                if np.isnan(self.last_best_values[k]):
                    self.last_best_values[k] = logs[k]
                else:
                    if logs[k] < self.last_best_values[k]:
                        register = False
                        break
            if register:
                for k,v in self.thresholds.items():
                    self.last_best_values[k] = logs[k]
                base_name = f'{self.model.name}_epoch_{str(epoch)}_{"_".join(["{}_{:.3f}".format(k,v) for k,v in self.last_best_values.items()])}'
                self.model.save(f'{base_name}.h5')                
                history_df = pd.DataFrame(self.model.history.history) 
                history_df.to_csv(f'{base_name}_history.csv',header=True, index=False)
                
                if self.colab_download:
                    from google.colab import files
                    files.download(f'{base_name}.h5')
                    files.download(f'{base_name}_history.csv')
                

In [ ]:
# data extraction
def getE2eData13(shuffle=False):
    hf_Train = h5.File(f'{data_location}/e2e_Train_Data_1000.h5', 'r')
    hf_Test = h5.File(f'{data_location}/e2e_Test_Data_1000.h5', 'r')

    X_train, Y_train = getNpArrayFromH5(hf_Train)
    X_test, Y_test = getNpArrayFromH5(hf_Test)
    Y_train = to_categorical(Y_train, 13)  # Process the label of tain
    Y_test = to_categorical(Y_test, 13)  # Process the label of te

    if shuffle:
        X_train, Y_train = coShuffled_vectors(X_train, Y_train)
        X_test, Y_test = coShuffled_vectors(X_test, Y_test)

    hf_Val = h5.File(f'{data_location}/e2e_Val_Data_1000.h5', 'r')
    X_validation, Y_validation = getNpArrayFromH5(hf_Val)
    Y_validation = to_categorical(Y_validation, 13)  

    return X_train, Y_train, X_test, Y_test, X_validation, Y_validation


def getE2eDataJustSecondary(shuffle=False,isColab=False):
    hf_Train = h5.File(f'{data_location}/e2e_Train_just_Secondary_Data_1000.h5', 'r')
    hf_Test = h5.File(f'{data_location}/e2e_Test_just_Secondary_Data_1000.h5', 'r')

    X_train, Y_train = getNpArrayFromH5(hf_Train)
    X_test, Y_test = getNpArrayFromH5(hf_Test)
    
    Y_train = to_categorical(Y_train, Y_test.shape[-1])  
    Y_test = to_categorical(Y_test, Y_test.shape[-1])  

    if shuffle:
        X_train, Y_train = coShuffled_vectors(X_train, Y_train)
        X_test, Y_test = coShuffled_vectors(X_test, Y_test)

    hf_Val = h5.File(f'{data_location}/e2e_Val_just_Secondary_Data_1000.h5', 'r')
    
    X_validation, Y_validation = getNpArrayFromH5(hf_Val)
    Y_validation = to_categorical(Y_validation, Y_test.shape[-1])  # Process the label of tain

    return X_train, Y_train, X_test, Y_test, X_validation, Y_validation


def getTest12Data():
    hf_Test = h5.File(f'{data_location}/e2e_Test_Data_1000_12classes.h5', 'r')
    X_test, Y_test = getNpArrayFromH5(hf_Test)
    Y_test = to_categorical(Y_test, 13)  # Process the label of te
    return X_test, Y_test


def get88KData():
    hf_Train = h5.File(f'{data_location}/e2e_Train_Data_1000_88.h5', 'r')
    hf_Test = h5.File(f'{data_location}/e2e_Test_Data_1000_88.h5', 'r')

    X_train, Y_train = getNpArrayFromH5(hf_Train)
    X_test, Y_test = getNpArrayFromH5(hf_Test)
    Y_train = to_categorical(Y_train, Y_test.shape[-1])  # Process the label of tain
    Y_test = to_categorical(Y_test, Y_test.shape[-1])  # Process the label of te

    hf_Val = h5.File(f'{data_location}/e2e_Val_Data_1000_88.h5', 'r') 
    X_validation, Y_validation = getNpArrayFromH5(hf_Val)
    Y_validation = to_categorical(Y_validation, Y_test.shape[-1])  # Process the label of tain

    return X_train, Y_train, X_test, Y_test, X_validation, Y_validation
    


A function that will take a model construction function (with _model_name_ and _input_shape_ arguments), train and fit it using the supplied data and callbacks. The _kwargs_ are supplied to the model fit function.

In [ ]:
from tensorflow.keras.callbacks import CSVLogger
import datetime

def compile_and_fit_model_basic(  model_func,
                                  model_name,
                                  input_shape,
                                  X_train,
                                  Y_train,
                                  save_max_epoch=True,
                                  save_final=False,
                                  patience_count = None,
                                  early_stopping_obs = 'val_sparse_categorical_accuracy',
                                  log_history = True,
                                  verbose_level = 0,
                                  **kwargs):
    m = None
    if isinstance(model_func, tf.keras.models.Model):
        m = model_func
        m._name = model_name
    else:
        m = model_func(model_name, input_shape)
      
    if 'validation_data' not in kwargs.keys() and 'val_' in early_stopping_obs:
        early_stopping_obs = early_stopping_obs.replace('val_','')

    callbacks_used = []
    if save_max_epoch:
        callbacks_used.append(ModelCheckpoint(f'/content/drive/MyDrive/data_papers/{paper_name}/model_checkpoints/{m.name}' + '_model_{epoch:03d}_{accuracy:0.3f}',
                                              save_weights_only=False,
                                              monitor = early_stopping_obs,
                                              mode='max',
                                              save_best_only=True))
    if patience_count is not None:
        callbacks_used.append(tf.keras.callbacks.EarlyStopping(monitor=early_stopping_obs, patience=patience_count))

    if log_history:
        callbacks_used.append(tf.keras.callbacks.CSVLogger(f"/content/drive/MyDrive/data_papers/{paper_name}/model_history/history_log_{model_name}_{datetime.date.today().strftime('%Y%m%d')}.csv", append=True))

    m.compile(loss=tf.keras.losses.CategoricalCrossentropy(), 
              optimizer=tf.keras.optimizers.Adam(), 
              metrics=[tf.keras.metrics.CategoricalAccuracy()])
    
    history = m.fit(X_train, Y_train, callbacks=callbacks_used, verbose=verbose_level, **kwargs)
    if save_final:
        make_dir_if_not_exist(model_name)
        m.save(f"/content/drive/MyDrive/data_papers/{paper_name}/model_finals/{m.name}_saved_model_after_fit")  # Save the model
    return (m, history)

A function that given a model or model directory create a new model up to the _layer_name_, then write the features matching the supplied _X_ and _Y_ as numpy arrays to google drive.

In [ ]:
def write_features_from_models(
        model_entry,
        layer_name,
        X_input, Y_input,
        reverse_one_hot=False,
        normalize_X_func=None,
        dataset_id = "NA",
        **kwargs):
  
    Y_new = None
    X_new = None
    if reverse_one_hot:
        Y_new = np.apply_along_axis(np.argmax, 1, Y_input) + 1
    else:
        Y_new = Y_input.copy()

    model_here = None
    if isinstance(model_entry, tf.keras.models.Model):
        model_here = model_entry
        model_file_name = model_here.name
    else:
        model_here = tf.keras.models.load_model(model_entry,**kwargs) 

    features_model = Model(model_here.input,
                            get_layer_by_name(model_here.layers, layer_name).output)
    if normalize_X_func is None:
        X_new = np.array(features_model.predict(X_input), dtype='float64')
    else:
        X_new = np.array(normalize_X_func(features_model.predict(X_input)), dtype='float64')

    np.save(f"/content/drive/MyDrive/data_papers/{paper_name}/model_features/{model_here.name}_features_{layer_name}_{dataset_id}_X", X_new, 
               allow_pickle=True, 
               fix_imports=True)
    np.save(f"/content/drive/MyDrive/data_papers/{paper_name}/model_features/{model_here.name}_features_{layer_name}_{dataset_id}_Y", Y_new, 
               allow_pickle=True, 
               fix_imports=True)


def write_features_from_models_idxCnt(
        model_entry,
        idxCnt,
        X_input, Y_input,
        reverse_one_hot=False,
        normalize_X_func=None,
        dataset_id = "NA",
        **kwargs):
  
    Y_new = None
    X_new = None
    if reverse_one_hot:
        Y_new = np.apply_along_axis(np.argmax, 1, Y_input) + 1
    else:
        Y_new = Y_input.copy()

    model_here = None
    if isinstance(model_entry, tf.keras.models.Model):
        model_here = model_entry
        model_file_name = model_here.name
    else:
        model_here = tf.keras.models.load_model(model_entry,**kwargs) 

    features_model = Model(model_here.input,
                            model_here.layers[idxCnt].output)
    if normalize_X_func is None:
        X_new = np.array(features_model.predict(X_input), dtype='float64')
    else:
        X_new = np.array(normalize_X_func(features_model.predict(X_input)), dtype='float64')

    np.save(f"/content/drive/MyDrive/data_papers/{paper_name}/model_features/{model_here.name}_features_{model_here.layers[idxCnt].name}_{dataset_id}_X", X_new, 
               allow_pickle=True, 
               fix_imports=True)
    np.save(f"/content/drive/MyDrive/data_papers/{paper_name}/model_features/{model_here.name}_features_{model_here.layers[idxCnt].name}_{dataset_id}_Y", Y_new, 
               allow_pickle=True, 
               fix_imports=True)               

Some functions to get scores on the results

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import re

def get_confusion_matrix_classification(model, X, Y_true):
    y_pred = model.predict(X)
    y_true = np.apply_along_axis(np.argmax, 1, Y_true)
    y_pred = np.apply_along_axis(np.argmax, 1, y_pred)
    return (confusion_matrix(y_true, y_pred), y_pred, y_true)

def construct_confusion_matrix(X, Y_true, Y_pred):
    y_true = Y_true
    y_pred = np.apply_along_axis(np.argmax, 1, Y_pred)
    return (confusion_matrix(y_true, y_pred), y_pred, y_true)

def pr_rc_f1_acc_from_supplied(y_pred, y_true):  
    pr, rc, f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")   
    acc = accuracy_score(y_true, y_pred)
    return pr, rc, f1, acc


In [ ]:
import re
import os

def dir_has_file_with_regex(dir_name, regex_string):
  filenames = [ f"{dir_name}/{dir_entry.name}" for dir_entry in os.scandir(dir_name) if os.path.isfile(f"{dir_name}/{dir_entry.name}") ]   
  filenames = [ fn for fn in filenames if re.match(regex_string, fn, re.IGNORECASE) ]
  return filenames



In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import  Dense, Flatten, Activation, Dropout, Embedding, Conv1D, Conv2D, MaxPooling2D, MaxPooling1D, Concatenate, BatchNormalization, GaussianNoise
from tensorflow.keras.layers import LSTM, TimeDistributed, Permute, Reshape, Lambda, RepeatVector, Input, Multiply, SimpleRNN, GRU, LeakyReLU
from keras_self_attention import SeqSelfAttention, SeqWeightedAttention
import site
import pandas as pd
import os
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
import datetime
import re

In [ ]:
X_train_1000e, Y_train_1000e, X_test_1000e, Y_test_1000e, X_val_1000e, Y_val_1000e = getE2eData13()
X_new_train = np.concatenate( (X_train_1000e, X_val_1000e), axis=0 )
Y_new_train = np.concatenate( (Y_train_1000e, Y_val_1000e), axis=0 )

# Show individual CNN/RNN

In [ ]:
os.listdir(f"/content/drive/MyDrive/data_papers/{paper_name}/summary_results/")

['temp',
 'individual_cnn_summary_20211206214644.npy',
 'individual_rnn_summary_20211207160708.npy',
 'ensemble_rnn_test_results_20211207213341.csv',
 'ensemble_cnn_test_results_20211207215955.csv',
 'cnn_rnn_ensemble_contour_data_20211207233449.csv',
 'cnn_rnn_ensemble_contour_data_20211207235706.csv']

In [ ]:
indiv_cnns = np.load(f"/content/drive/MyDrive/data_papers/{paper_name}/summary_results/individual_cnn_summary_20211206214644.npy")
indiv_rnns = np.load(f"/content/drive/MyDrive/data_papers/{paper_name}/summary_results/individual_rnn_summary_20211207160708.npy")

In [ ]:
indiv_cnns.mean(axis=0)

array([0.95071833, 0.94772727, 0.947997  , 0.94772727])

In [ ]:
indiv_rnns.mean(axis=0)

array([0.77462007, 0.77233877, 0.75532836, 0.77233877])

In [ ]:
indiv_rnns.shape

(12, 4)

In [ ]:
indiv_rnns

array([[0.81086121, 0.78088578, 0.7751795 , 0.78088578],
       [0.80863864, 0.78554779, 0.77521623, 0.78554779],
       [0.91999956, 0.91491841, 0.91281406, 0.91491841],
       [0.86983951, 0.86829837, 0.86383539, 0.86829837],
       [0.92132802, 0.91025641, 0.91037113, 0.91025641],
       [0.92490678, 0.91958042, 0.918416  , 0.91958042],
       [0.18672943, 0.23776224, 0.12371224, 0.23776224],
       [0.93097164, 0.92657343, 0.92717165, 0.92657343],
       [0.86647343, 0.85198135, 0.84579051, 0.85198135],
       [0.25692532, 0.29020979, 0.23698688, 0.29020979],
       [0.89440603, 0.88111888, 0.87574705, 0.88111888],
       [0.90436124, 0.9009324 , 0.89869963, 0.9009324 ]])

# Show ensemble CNN/RNN/CNN&RNN

In [ ]:
os.listdir(f"/content/drive/MyDrive/data_papers/{paper_name}/summary_results/")

['temp',
 'individual_cnn_summary_20211206214644.npy',
 'individual_rnn_summary_20211207160708.npy',
 'ensemble_rnn_test_results_20211207213341.csv',
 'ensemble_cnn_test_results_20211207215955.csv',
 'cnn_rnn_ensemble_contour_data_20211207233449.csv',
 'cnn_rnn_ensemble_contour_data_20211207235706.csv']

In [ ]:
ensemble_rnns = pd.read_csv(f"/content/drive/MyDrive/data_papers/{paper_name}/summary_results/ensemble_rnn_test_results_20211207213341.csv")
ensemble_cnns = pd.read_csv(f"/content/drive/MyDrive/data_papers/{paper_name}/summary_results/ensemble_cnn_test_results_20211207215955.csv")
ensemble_cnnrnnss = pd.read_csv(f"/content/drive/MyDrive/data_papers/{paper_name}/summary_results/cnn_rnn_ensemble_contour_data_20211207235706.csv")

In [ ]:
ensemble_cnns["CNN%"] = np.select([ensemble_cnns.Type=="CNN256",ensemble_cnns.Type=="RNN"],[1.0,0.0])
ensemble_rnns["CNN%"] = np.select([ensemble_rnns.Type=="CNN256",ensemble_rnns.Type=="RNN"],[1.0,0.0])


In [ ]:
ensemble_cnnrnnss = ensemble_cnnrnnss.drop_duplicates()
ensemble_cnnrnnss["NumOfModels"] = ensemble_cnnrnnss.NumOfA + ensemble_cnnrnnss.NumOfB
ensemble_cnnrnnss["Type"] = "CNN_RNN"
ensemble_cnnrnnss["CNN%"] = ensemble_cnnrnnss.NumOfA/ensemble_cnnrnnss["NumOfModels"]

In [ ]:
# ensemble_rnns

In [ ]:
ensemble_data = pd.concat([ ensemble_rnns[["Type","Data","NumOfModels","CNN%","Pr","Rc","F1","Acc"]],
            ensemble_cnns[["Type","Data","NumOfModels","CNN%","Pr","Rc","F1","Acc"]],
            ensemble_cnnrnnss[["Type","Data","NumOfModels","CNN%","Pr","Rc","F1","Acc"]] ]).drop_duplicates()

In [ ]:
ensemble_data.loc[(ensemble_data["CNN%"] >= 1.0) & (ensemble_data["NumOfModels"]>=20),["NumOfModels","Pr","Rc","F1","Acc"]]

,NumOfModels,Pr,Rc,F1,Acc
70,20,0.960386,0.959207,0.959405,0.959207
71,20,0.956702,0.955711,0.955703,0.955711
72,22,0.957807,0.956876,0.957092,0.956876
73,22,0.963952,0.962704,0.962990,0.962704
74,25,0.964572,0.963869,0.964025,0.963869
75,25,0.963641,0.962704,0.962919,0.962704
76,30,0.963598,0.962704,0.962878,0.962704
77,30,0.960497,0.959207,0.959534,0.959207
78,35,0.961246,0.960373,0.960547,0.960373
79,35,0.965308,0.963869,0.964177,0.963869


In [ ]:
( np.array(ensemble_data.loc[(ensemble_data["CNN%"] >= 1.0) & (ensemble_data["NumOfModels"]==20),["NumOfModels","Pr","Rc","F1","Acc"]].mean(axis=0)) , 
  np.array(ensemble_data.loc[(ensemble_data["CNN%"] >= 1.0) & (ensemble_data["NumOfModels"]==40),["NumOfModels","Pr","Rc","F1","Acc"]].mean(axis=0)) )

(array([20.        ,  0.95854428,  0.95745921,  0.95755402,  0.95745921]),
 array([40.        ,  0.95979251,  0.95862471,  0.95883819,  0.95862471]))

In [ ]:
np.array(ensemble_data.loc[(ensemble_data["CNN%"] == 0.0) & (ensemble_data["NumOfModels"]==12),["NumOfModels","Pr","Rc","F1","Acc"]]).mean(axis=0)

array([12.        ,  0.92936153,  0.92657343,  0.92470842,  0.92657343])

In [ ]:
np.array(ensemble_data.loc[(ensemble_data["CNN%"] == 0.5) & (ensemble_data["NumOfModels"]==12),["NumOfModels","Pr","Rc","F1","Acc"]]).mean(axis=0)

array([12.        ,  0.96461972,  0.96386946,  0.96385723,  0.96386946])

# Show fully integrated models

In [ ]:
os.listdir(f"/content/drive/MyDrive/data_papers/{paper_name}/model_finals")

['CNN256_A_1_20211124110828_saved_model_after_fit',
 'CNN256_A_2_20211124111459_saved_model_after_fit',
 'CNN256_A_3_20211124112133_saved_model_after_fit',
 'CNN256_A_4_20211124112755_saved_model_after_fit',
 'CNN256_A_5_20211124113440_saved_model_after_fit',
 'CNN256_A_6_20211124114114_saved_model_after_fit',
 'CNN256_A_7_20211124114753_saved_model_after_fit',
 'CNN256_A_8_20211124115418_saved_model_after_fit',
 'CNN256_A_9_20211124120100_saved_model_after_fit',
 'CNN256_A_10_20211124120744_saved_model_after_fit',
 'CNN256_A_11_20211124121436_saved_model_after_fit',
 'CNN256_A_12_20211124122043_saved_model_after_fit',
 'CNN256_A_13_20211124122718_saved_model_after_fit',
 'CNN256_A_14_20211124123410_saved_model_after_fit',
 'CNN256_A_15_20211124124025_saved_model_after_fit',
 'CNN256_A_16_20211124124717_saved_model_after_fit',
 'CNN256_A_17_20211124125352_saved_model_after_fit',
 'CNN256_A_18_20211124130025_saved_model_after_fit',
 'CNN256_A_19_20211124130651_saved_model_after_fit',
 '

In [ ]:
cnn20_par = load_model(f"/content/drive/MyDrive/data_papers/{paper_name}/model_finals/ParallelCNN_A_20211209020220_saved_model_after_fit")



In [ ]:
cnn2_par = load_model(f"/content/drive/MyDrive/data_papers/{paper_name}/model_finals/ParallelCNN_A2_20211209133850_saved_model_after_fit")

In [ ]:
cnn1rnn1_par = load_model(f"/content/drive/MyDrive/data_papers/{paper_name}/model_finals/ParallelCNNRNN_A2_20211210221624_saved_model_after_fit")

In [ ]:
cnn20_par_y_predict_here = np.array(cnn20_par.predict(X_test_1000e), dtype='float64')
cnn20_par_y_predict_here = np.apply_along_axis(np.argmax, 1, cnn20_par_y_predict_here)
pr_rc_f1_acc_from_supplied(cnn20_par_y_predict_here, np.apply_along_axis(np.argmax, 1, Y_test_1000e))


(0.9318062855471186,
 0.9184149184149184,
 0.9195749976697266,
 0.9184149184149184)

In [ ]:
cnn2_par_y_predict_here = np.array(cnn2_par.predict(X_test_1000e), dtype='float64')
cnn2_par_y_predict_here = np.apply_along_axis(np.argmax, 1, cnn2_par_y_predict_here)
pr_rc_f1_acc_from_supplied(cnn2_par_y_predict_here, np.apply_along_axis(np.argmax, 1, Y_test_1000e))

(0.9346676642409038,
 0.9289044289044289,
 0.9267679288900833,
 0.9289044289044289)

In [ ]:
cnn1rnn1_par_y_predict_here = np.array(cnn1rnn1_par.predict(X_test_1000e), dtype='float64')
cnn1rnn1_par_y_predict_here = np.apply_along_axis(np.argmax, 1, cnn1rnn1_par_y_predict_here)
pr_rc_f1_acc_from_supplied(cnn1rnn1_par_y_predict_here, np.apply_along_axis(np.argmax, 1, Y_test_1000e))

(0.9537472379437839, 0.951048951048951, 0.9517260643724785, 0.951048951048951)